In [ ]:
!pip3 install gym
# !pip3 install stable-baselines3[extra]
# !pip3 install 'shimmy>=2.0'

## Gym Class

In [1]:
import gym
from gym import spaces
import numpy as np
import pybullet as p
import pybullet_data
import math
import time
import random


class PandaTennisEnv(gym.Env):
    def __init__(self, enable_gui = True):
        super(PandaTennisEnv, self).__init__()
        
        # preset parameters
        self.time_step = 1.0 / 240.0
        #ball properties
        self.ball_radius = 0.0335  # Radius (6.7 cm diameter / 2)
        self.ball_mass = 0.058  # Standard tennis ball mass in kg
        self.ball_color = [1, 1, 0, 1]
        self.ball_position = [0.5, 0.5, 10]  # Initial position of the ball
        self.ball_initial_vel = [0, 0, 10]
        self.lateral_friction = 0.5
        self.restitution = 0.8

        
        # Wall properties
        self.wall_size = [0.55, 1.5, 1.0]  # [length, thickness, height]
        self.wall_mass = 2000  # Static wall
        self.wall_color = [0.5, 0.5, 0.5, 1]
        self.wall_position = [5, 0.0, 0.0]  # Adjust the position as needed
        self.wall_orientation = p.getQuaternionFromEuler([0, 0, 0])  # No rotation


        # Table dimensions and properties
        self.table_length = 2.74  # meters
        self.table_width = 1.525  # meters
        self.table_height = 0.76  # meters
        self.table_position = [2, 0, self.table_height / 2]  # Move table to (2, 2)
        self.table_restitution = 0.8
        self.table_friction = 0.5

        ## How long should the system rest to adjust joints
        self.wait_period = 40
        self.step_number = 0
        # Define observation and action spaces
        # Define the action space with specific ranges
        husky_low = np.array([-5, -5, -5, -5], dtype=np.float32)  # Husky wheels: -5 to +5
        husky_high = np.array([5, 5, 5, 5], dtype=np.float32)
        panda_low = np.radians([-3, -3, -3, -3, -3, -3, -3])  # Panda joints: -180 degrees
        panda_high = np.radians([3, 3, 3, 3, 3, 3, 3])  # Panda joints: +180 degrees
        low = np.concatenate([panda_low, husky_low])
        high = np.concatenate([panda_high, husky_high])


        
        # Connect to PyBullet in GUI mode
        self.physics_client = p.connect(p.GUI if enable_gui else p.DIRECT)
        p.setAdditionalSearchPath(pybullet_data.getDataPath())
        self.initialize_everything()
        
        
        self.action_space = spaces.Box(low=low, high=high, dtype=np.float32)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(49,), dtype=np.float32)

    def initialize_everything(self):
        p.setTimeStep(self.time_step)
        p.setGravity(0,0,-9.81)
        self.step_number = 0
        self.plane_id = p.loadURDF("plane.urdf")
        self.husky_id = p.loadURDF("husky/husky.urdf", [0, 0, 0])
        self.panda_id = p.loadURDF("franka_panda_mod/panda.urdf", [0.2, 0, 0.30])
        self.bat_id = p.loadURDF("Squash.urdf", basePosition=[0, 0, 0.8], baseOrientation = p.getQuaternionFromEuler([0, 0, 0]))  # Replace with the actual path to the URDF

        
        # Create wall collision shape
        wall_collision = p.createCollisionShape(p.GEOM_BOX, halfExtents=self.wall_size)
        # Create wall visual shape
        wall_visual = p.createVisualShape(p.GEOM_BOX, halfExtents=self.wall_size, rgbaColor=self.wall_color)  # Grey color
        # Create the wall body
        self.wall_id = p.createMultiBody(
            baseMass=self.wall_mass,
            baseCollisionShapeIndex=wall_collision,
            baseVisualShapeIndex=wall_visual,
            basePosition=self.wall_position,
            baseOrientation=self.wall_orientation
        )
        p.changeDynamics(self.wall_id, -1, lateralFriction=self.lateral_friction, restitution=self.restitution)
        self.mount_bodies()
        p.changeDynamics(self.bat_id, -1, restitution=self.restitution, lateralFriction=self.lateral_friction)
        p.changeDynamics(
            bodyUniqueId=self.plane_id,
            linkIndex=-1,  # -1 refers to the base link (plane has no other links)
            lateralFriction=self.lateral_friction,
            restitution=self.restitution
        )
        # Apply dynamics properties to all links of panda
        num_joints = p.getNumJoints(self.panda_id)
        for link_index in range(-1, num_joints):  # -1 refers to the base link
            p.changeDynamics(self.panda_id, link_index, restitution=self.restitution, lateralFriction=self.lateral_friction)

        # Apply damping for all joints
        for joint_index in range(num_joints):
            p.changeDynamics(self.panda_id, joint_index, linearDamping=0.04, angularDamping=0.04)

        
        self.get_new_ball([random.uniform(-1,1), random.uniform(-1,1), random.uniform(5,7)])
        self.throw_ball(self.ball_initial_vel)

        ## build table
        self.build_table()


    def build_table(self):
        # Create the table as a box
        table_visual = p.createVisualShape(
            shapeType=p.GEOM_BOX,
            halfExtents=[self.table_length / 2, self.table_width / 2, self.table_height / 2],
            rgbaColor=[0, 0.6, 0, 1],  # Dark green color for the table
        )
        
        table_collision = p.createCollisionShape(
            shapeType=p.GEOM_BOX,
            halfExtents=[self.table_length / 2, self.table_width / 2, self.table_height / 2],
        )
        
        self.table_id = p.createMultiBody(
            baseMass=0,
            baseCollisionShapeIndex=table_collision,
            baseVisualShapeIndex=table_visual,
            basePosition=self.table_position,
        )
        
        # Set the physical properties of the table
        p.changeDynamics(self.table_id, -1, restitution=self.table_restitution, lateralFriction=self.table_friction)
        
        # Draw the court lines
        line_color = [1, 1, 1, 1]  # White color for the lines
        
        # Outer border (2 cm width)
        p.addUserDebugLine(
            lineFromXYZ=[self.table_position[0] - self.table_length / 2, self.table_position[1] - self.table_width / 2, self.table_height],
            lineToXYZ=[self.table_position[0] - self.table_length / 2, self.table_position[1] + self.table_width / 2, self.table_height],
            lineColorRGB=line_color,
            lineWidth=4,
        )
        p.addUserDebugLine(
            lineFromXYZ=[self.table_position[0] - self.table_length / 2, self.table_position[1] + self.table_width / 2, self.table_height],
            lineToXYZ=[self.table_position[0] + self.table_length / 2, self.table_position[1] + self.table_width / 2, self.table_height],
            lineColorRGB=line_color,
            lineWidth=4,
        )
        p.addUserDebugLine(
            lineFromXYZ=[self.table_position[0] + self.table_length / 2, self.table_position[1] + self.table_width / 2, self.table_height],
            lineToXYZ=[self.table_position[0] + self.table_length / 2, self.table_position[1] - self.table_width / 2, self.table_height],
            lineColorRGB=line_color,
            lineWidth=4,
        )
        p.addUserDebugLine(
            lineFromXYZ=[self.table_position[0] + self.table_length / 2, self.table_position[1] - self.table_width / 2, self.table_height],
            lineToXYZ=[self.table_position[0] - self.table_length / 2, self.table_position[1] - self.table_width / 2, self.table_height],
            lineColorRGB=line_color,
            lineWidth=4,
        )
        
        # Centerline (3 mm width)
        p.addUserDebugLine(
            lineFromXYZ=[self.table_position[0], self.table_position[1] - self.table_width / 2, self.table_height],
            lineToXYZ=[self.table_position[0], self.table_position[1] + self.table_width / 2, self.table_height],
            lineColorRGB=line_color,
            lineWidth=4,
        )
        
        p.addUserDebugLine(
            lineFromXYZ=[self.table_position[0] - self.table_length / 2, self.table_position[1], self.table_height],
            lineToXYZ=[self.table_position[0] + self.table_length / 2, self.table_position[1], self.table_height],
            lineColorRGB=line_color,
            lineWidth=4,
        )

    
    def mount_bodies(self):
        # mount the panda on the husky
        husky_chassis_link_index = 0  # Replace with appropriate link index
        
        # Create the fixed constraint
        self.husky_panda_mount = p.createConstraint(
            parentBodyUniqueId=self.husky_id,
            parentLinkIndex=husky_chassis_link_index,
            childBodyUniqueId=self.panda_id,
            childLinkIndex=-1,  # -1 means the base link of the child
            jointType=p.JOINT_FIXED,
            jointAxis=[0, 0, 0],
            parentFramePosition=[0.3, 0, 0.25],  # Offset on Husky where the Panda arm will be attached
            childFramePosition=[0, 0, 0]  # Offset on Panda where it attaches to Husky
        )

        # set panda to preset position
        preset_position = [0, -0.810, 0, -1.52, 0.219, 0.818, -1.1]
        for joint_index, target_position in zip(range(0, 7), preset_position):
            p.setJointMotorControl2(
                bodyUniqueId=self.panda_id,
                jointIndex=joint_index,
                controlMode=p.POSITION_CONTROL,
                targetPosition=target_position,
                force=500  # Adjust max torque/force as needed
            )

        for i in range (self.wait_period):
            p.stepSimulation()
        # panda_bat_mount
        end_effector_index = 7  # Replace with your actual end effector index
        
        # Position and orientation offsets to align the tennis bat with the end effector
        offset_position = [0, 0, 0.25]  # Adjust based on your setup
        offset_orientation = p.getQuaternionFromEuler([0, 0, 0])  # Adjust if needed
        
        # Create a fixed constraint
        self.panda_bat_mount = p.createConstraint(
            parentBodyUniqueId=self.panda_id,
            parentLinkIndex=end_effector_index,
            childBodyUniqueId=self.bat_id,
            childLinkIndex=-1,  # Base of the tennis bat
            jointType=p.JOINT_FIXED,
            jointAxis=[0, 0, 0],
            parentFramePosition=offset_position,
            parentFrameOrientation=offset_orientation,
            childFramePosition=[0, 0, 0],
            childFrameOrientation=[0, 0, 0]
        )
        for i in range (self.wait_period):
            p.stepSimulation()
        
    def reset(self):
        # Reset simulation
        p.resetSimulation()
        self.initialize_everything()
        return self._get_observation()
    
    def _get_observation(self):

        # ball states
        ball_pos, ball_orn = p.getBasePositionAndOrientation(self.ball_id)
        ball_linear_velocity, ball_angular_velocity = p.getBaseVelocity(self.ball_id) # Ignore angular velocity. Difficult to estimate in reality
        

        # bat states
        bat_pos, bat_orn = p.getBasePositionAndOrientation(self.bat_id)
        bat_dynamics_info = p.getDynamicsInfo(self.bat_id, -1)
        bat_com_offset = bat_dynamics_info[3]  # Local COM position
        bat_com_global = p.multiplyTransforms(bat_pos, bat_orn, bat_com_offset, [0, 0, 0, 1])[0] # Global COM position


        # Husky states
        husky_pos, husky_orn = p.getBasePositionAndOrientation(self.husky_id)
        husky_lin_vel, husky_ang_vel = p.getBaseVelocity(self.husky_id)
        
        

        # Panda states
        panda_num_joints = p.getNumJoints(self.panda_id)
        panda_joint_states = p.getJointStates(self.panda_id, range(panda_num_joints))
        panda_joint_pos = [state[0] for state in panda_joint_states]
        panda_joint_vel = [state[1] for state in panda_joint_states]
        
        system_state = np.concatenate([
                np.array(ball_pos),                 # 3 values: Ball position
                np.array(ball_orn),                # 4 values: Ball orientation
                np.array(ball_linear_velocity),    # 3 values: Ball linear velocity
                np.array(bat_pos),                 # 3 values: Bat position
                np.array(bat_orn),                 # 4 values: Bat orientation
                np.array(bat_com_global),          # 3 values: Bat COM global position
                np.array(husky_pos),               # 3 values: Husky position
                np.array(husky_orn),               # 4 values: Husky orientation
                np.array(husky_lin_vel),           # 3 values: Husky linear velocity
                np.array(husky_ang_vel),           # 3 values: Husky angular velocity
                np.array(panda_joint_pos),         # 7 values: Panda joint positions
                np.array(panda_joint_vel)          # 7 values: Panda joint velocities
            ])
        # system_state = {
        #         "ball_position": ball_pos,
        #         "ball_orientation": ball_orn,
        #         "ball_linear_velocity": ball_linear_velocity,
        #         "bat_position": bat_pos,
        #         "bat_orientation": bat_orn,
        #         "bat_com_global": bat_com_global,
        #         "husky_position": husky_pos,
        #         "husky_orientation": husky_orn,
        #         "husky_linear_velocity": husky_lin_vel,
        #         "husky_angular_velocity": husky_ang_vel,
        #         "panda_joint_positions": panda_joint_pos,
        #         "panda_joint_velocities": panda_joint_vel
        #     }
        
        return system_state


    def get_new_ball(self, initial_position):
        # Create ball collision shape
        ball_collision = p.createCollisionShape(p.GEOM_SPHERE, radius=self.ball_radius)
        # Create ball visual shape
        ball_visual = p.createVisualShape(p.GEOM_SPHERE, radius=self.ball_radius, rgbaColor=self.ball_color)  # Yellow color
        # Create the ball body
        self.ball_id = p.createMultiBody(
            baseMass=self.ball_mass,
            baseCollisionShapeIndex=ball_collision,
            baseVisualShapeIndex=ball_visual,
            basePosition=initial_position,
        )
        p.changeDynamics(self.ball_id, -1, lateralFriction=self.lateral_friction, restitution=self.restitution)

    # Function to draw the COM of bat and axes
    def visualize_com(self):
        # Function to rotate a vector by a quaternion
        def rotate_vector(vector, quaternion):
            return p.multiplyTransforms([0, 0, 0], quaternion, vector, [0, 0, 0, 1])[0]
        
        # Get COM for the base link
        dynamics_info = p.getDynamicsInfo(self.bat_id, -1)
        com_offset = dynamics_info[3]  # Local COM position
        base_pos, base_orn = p.getBasePositionAndOrientation(self.bat_id)
        
        # Transform the local COM to global coordinates
        com_global = p.multiplyTransforms(base_pos, base_orn, com_offset, [0, 0, 0, 1])[0]
        
        # Define local unit axes
        axis_length = 0.2  # Length of the axes
        x_axis = [axis_length, 0, 0]
        y_axis = [0, axis_length, 0]
        z_axis = [0, 0, axis_length]
        
        # Rotate local axes to match the base's orientation
        x_axis_global = rotate_vector(x_axis, base_orn)
        y_axis_global = rotate_vector(y_axis, base_orn)
        z_axis_global = rotate_vector(z_axis, base_orn)
        
        # Compute global endpoints of the axes
        x_end = [com_global[i] + x_axis_global[i] for i in range(3)]
        y_end = [com_global[i] + y_axis_global[i] for i in range(3)]
        z_end = [com_global[i] + z_axis_global[i] for i in range(3)]
        
        # Draw lines for the axes
        p.addUserDebugLine(com_global, x_end, [1, 0, 0], 3)  # Red for X-axis
        p.addUserDebugLine(com_global, y_end, [0, 1, 0], 3)  # Green for Y-axis
        p.addUserDebugLine(com_global, z_end, [0, 0, 1], 3)  # Blue for Z-axis
        

    # remove all the lines or user debug texts added
    def removeUserDebugItems(self):
        p.removeAllUserDebugItems()

    
    def throw_ball(self, initial_velocity):
        # Throw the ball with initial velocity
        # initial_velocity = [3, 0, 2]  # Velocity in x, y, z directions
        p.resetBaseVelocity(self.ball_id, linearVelocity=initial_velocity)


    def step(self, action):
        self.step_number += 1
        # Split actions for Panda and Husky
        panda_actions = action[:7]  # First 7 values for Panda arm
        husky_actions = action[7:]  # Last 4 values for Husky wheels
    
        # Apply actions to Panda arm
        # for i, act in enumerate(panda_actions):
        #     p.setJointMotorControl2(
        #         self.panda_id,
        #         i,
        #         p.POSITION_CONTROL,
        #         targetPosition=act,  # Directly use radians action
        #         force=500
        #     )

        # # Set initial damping to prevent oscillations
        # for joint in range(len(panda_actions)):
        #     pb.setJointMotorControl2(
        #         bodyIndex=self.panda_id,
        #         jointIndex=joint,
        #         controlMode=p.VELOCITY_CONTROL,
        #         targetVelocity=0,
        #         gain=0.1  # Damping gain (adjust as needed)
        #     )


         # Apply actions to Husky wheels
        panda_joints = [0, 1, 2, 3, 4, 5, 6]  # Joint indices for the Husky wheels
        for i, act in enumerate(panda_actions):
            p.setJointMotorControl2(
                self.panda_id,
                panda_joints[i],
                p.VELOCITY_CONTROL,
                targetVelocity=act  # Use velocity directly within [-5, 5]
            )


        
        
        # Apply actions to Husky wheels
        wheel_joints = [2, 3, 4, 5]  # Joint indices for the Husky wheels
        for i, act in enumerate(husky_actions):
            p.setJointMotorControl2(
                self.husky_id,
                wheel_joints[i],
                p.VELOCITY_CONTROL,
                targetVelocity=act  # Use velocity directly within [-5, 5]
            )
    
        # Step the simulation
        p.stepSimulation()
    
        # Calculate reward and check if done
        reward = self._calculate_reward()
        done = self._check_done()

        ## Enable markers
        # self.visualize_com()
        # self.removeUserDebugItems()
        return self._get_observation(), reward, done, {}

    def ball_is_approaching(self):
        """
        Determine if a moving ball is approaching a point in 3D space.
    
        Parameters:
            ball_position (np.ndarray): The current position of the ball as a 3D vector (x, y, z).
            ball_velocity (np.ndarray): The velocity of the ball as a 3D vector (vx, vy, vz).
            target_point (np.ndarray): The target point in space as a 3D vector (px, py, pz).
    
        Returns:
            bool: True if the ball is approaching the target point, False otherwise.
        """
        ball_position, _ = p.getBasePositionAndOrientation(self.ball_id)
        ball_velocity, _ = p.getBaseVelocity(self.ball_id) # Ignore angular velocity. Difficult to estimate in reality
        bat_pos, bat_orn = p.getBasePositionAndOrientation(self.bat_id)
        bat_dynamics_info = p.getDynamicsInfo(self.bat_id, -1)
        bat_com_offset = bat_dynamics_info[3]  # Local COM position
        bat_com_global = p.multiplyTransforms(bat_pos, bat_orn, bat_com_offset, [0, 0, 0, 1])[0] # Global COM position
        target_point = bat_com_global
        # Calculate the relative position vector from the ball to the target
        relative_position = np.array(target_point) - np.array(ball_position)
        
        # Calculate the dot product of the relative position and velocity
        dot_product = np.dot(relative_position, ball_velocity)
        
        # If the dot product is negative, the ball is approaching the target point
        return dot_product < 0

    
    def _calculate_reward(self):
        total_reward = 0
    
        # Get Husky's current state
        husky_pos, husky_orn = p.getBasePositionAndOrientation(self.husky_id)
        husky_lin_vel, husky_ang_vel = p.getBaseVelocity(self.husky_id)
        
        # Husky Stability: penalize if husky is too tilted
        husky_pitch = np.arctan2(2 * (husky_orn[3] * husky_orn[0] + husky_orn[0] * husky_orn[1]),
                                 1 - 2 * (husky_orn[0]**2 + husky_orn[1]**2))
        husky_roll = np.arctan2(2 * (husky_orn[3] * husky_orn[1] + husky_orn[2] * husky_orn[0]),
                                1 - 2 * (husky_orn[1]**2 + husky_orn[2]**2))
        
        # Stability reward: higher penalty for larger tilt
        stability_penalty = np.clip(np.abs(husky_pitch) + np.abs(husky_roll), 0, 1)
        total_reward -= 0.5 * stability_penalty  # negative reward for instability
        
        # Panda Arm: track bat's position and orientation
        bat_pos, bat_orn = p.getBasePositionAndOrientation(self.bat_id)
        bat_dynamics_info = p.getDynamicsInfo(self.bat_id, -1)
        bat_com_offset = bat_dynamics_info[3]  # Local COM position
        bat_com_global = p.multiplyTransforms(bat_pos, bat_orn, bat_com_offset, [0, 0, 0, 1])[0]
        
        # Ball: get position and velocity of the ball
        ball_pos, ball_orn = p.getBasePositionAndOrientation(self.ball_id)
        ball_linear_velocity, _ = p.getBaseVelocity(self.ball_id)
        
        # Reward for the ball approaching the bat
        approaching_reward = 0
        if self.ball_is_approaching():
            approaching_reward = 10  # Reward when the ball is approaching the bat
        
        # Reward for successful hit (after intercepting)
        hit_reward = 0
        if np.linalg.norm(ball_linear_velocity) > 0 and ball_pos[2] > 0:  # Ball is hit and moving upwards
            hit_reward = 1  # Reward for hitting the ball upwards
        
        # Reward for bat position matching the ideal position to hit the ball
        bat_to_ball_dist = np.linalg.norm(np.array(ball_pos) - np.array(bat_com_global))
        bat_orientation_diff = np.linalg.norm(np.array(bat_orn) - np.array(ball_orn))  # You may calculate ideal bat orientation for hitting
    
        intercept_reward = -bat_to_ball_dist - bat_orientation_diff  # Reward for positioning bat for intercepting ball
        total_reward += 0.5 * intercept_reward + 0.5 * hit_reward + 0.5 * approaching_reward + 0.05*self.step_number # Adding rewards for interception, hit, and approach
        
        # Add more reward for consecutive hits (optional)
        consecutive_hits_reward = 0  # Could track in the class if the robot hits the ball multiple times in a row
        total_reward += consecutive_hits_reward
        return total_reward
    
    def _check_done(self):
        # Get the position of the ball
        ball_pos, ball_orn = p.getBasePositionAndOrientation(self.ball_id)
        
        # Get the vertical velocity of the ball (in the z direction)
        ball_linear_velocity, ball_angular_velocity = p.getBaseVelocity(self.ball_id)
        ball_vertical_velocity = ball_linear_velocity[2]  # The z component of velocity
        
        # Done condition: Ball hits the ground (z position below a threshold or vertical velocity down)
        if ball_pos[2] < 0.1:
            # Ball has hit the ground, episode should end
            return True
        
        return False
    
    def render(self, mode="human"):
        pass  # PyBullet GUI provides rendering
    
    def close(self):
        p.disconnect()


pybullet build time: Feb  4 2024 12:55:26


## Test Run

In [3]:
# Test the model
env = PandaTennisEnv(enable_gui = True)

/Users/sakibchowdhury/Library/Python/3.9/lib/python/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [ ]:
# Load the trained model
# model = PPO.load("panda_tennis_model")

# Test the model
# env = PandaTennisEnv()

obs = env.reset()
# env.get_new_ball([1, 0, 7])
# env.throw_ball([0, 0, 0])
for i in range(100000):
    # panda_action = [-0.066, -1.925, -0.033, -2.016, 0.099, 1.858, 1.818] # joint position
    panda_action = [-0.1, -0.1, -0.3, -0.1, 0.099, 0.1, 0.1] # panda_velocities
    husky_action = [1, 1, 1, 1] # wheel velocity
    action = panda_action + husky_action
    obs, reward, done, info = env.step(action)
    # print(obs)
    time.sleep(1/240)
    # husky_position, husky_orientation = p.getBasePositionAndOrientation(env.husky_id)
    # print(husky_position, husky_orientation)
    
    if done:
        obs = env.reset()


## Enable Sliders for Panda Joint Control

In [5]:
# Get joint information
num_joints = p.getNumJoints(env.panda_id)
joint_indices = [i for i in range(num_joints) if p.getJointInfo(env.panda_id, i)[2] == p.JOINT_REVOLUTE]

# Create sliders for each joint
sliders = {}
for joint_index in joint_indices:
    joint_info = p.getJointInfo(env.panda_id, joint_index)
    joint_name = joint_info[1].decode("utf-8")
    joint_limit_low = joint_info[8]
    joint_limit_high = joint_info[9]
    slider = p.addUserDebugParameter(joint_name, joint_limit_low, joint_limit_high, 0.0)
    sliders[joint_index] = slider

# Simulation loop
while True:
    # Read slider values and control joints
    for joint_index, slider in sliders.items():
        slider_value = p.readUserDebugParameter(slider)
        p.setJointMotorControl2(
            bodyUniqueId=env.panda_id,
            jointIndex=joint_index,
            controlMode=p.POSITION_CONTROL,
            targetPosition=slider_value,
            force=500  # Set maximum torque/force
        )
    
    # Step the simulation
    p.stepSimulation()
    time.sleep(0.01)  # Add a small delay for stability

error: Failed to read parameter.

## Make Panda Dance (don't use - panda uses velocity control now)

In [ ]:
# Get number of joints
num_joints = p.getNumJoints(env.panda_id)

# Define the range of motion for the joints
joint_ranges = {
    0: (-2.9, 2.9),
    1: (-1.76, 1.76),
    2: (-2.9, 2.9),
    3: (-3.07, -0.07),
    4: (-2.9, 2.9),
    5: (-0.017, 3.75),
    6: (-2.9, 2.9),
}

# Simulation parameters
duration = 100  # Total time for the dance (seconds)
frequency = 0.05  # Frequency of oscillation (Hz)

# Create a time-based loop
start_time = time.time()
while time.time() - start_time < duration:
    current_time = time.time() - start_time

    # Set joint positions in a sinusoidal pattern
    for joint_index in range(7):  # Panda has 7 controllable joints
        joint_range = joint_ranges[joint_index]
        joint_center = (joint_range[0] + joint_range[1]) / 2
        joint_amplitude = (joint_range[1] - joint_range[0]) / 4

        # Calculate target position using a sine wave
        target_position = joint_center + joint_amplitude * np.sin(2 * np.pi * frequency * current_time + joint_index)

        # Set joint position
        p.setJointMotorControl2(
            bodyUniqueId=env.panda_id,
            jointIndex=joint_index,
            controlMode=p.POSITION_CONTROL,
            targetPosition=target_position,
        )
    # Clear previous lines
    env.removeUserDebugItems()
    
    # Visualize the COM of the bat
    env.visualize_com()
    print(env._get_observation())

    # Step simulation
    p.stepSimulation()
    time.sleep(1 / 240)  # Match the simulation timestep (240 Hz)

In [3]:
pybullet_data.getDataPath()

'/home/sakib/.local/lib/python3.8/site-packages/pybullet_data'

## Experiment

In [2]:
# Test the model
env = PandaTennisEnv(enable_gui = True)

2025-01-27 14:20:26.579 Python[10941:13927298] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-27 14:20:26.579 Python[10941:13927298] +[IMKInputSession subclass]: chose IMKInputSession_Modern
/Users/sakibchowdhury/Library/Python/3.9/lib/python/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [ ]:
while True:
    # panda_action = [-0.066, -1.925, -0.033, -2.016, 0.099, 1.858, 1.818] # joint position
    panda_action = [0.0]*7 # panda_velocities
    husky_action = [1]*4 # wheel velocity
    action = panda_action + husky_action
    obs, reward, done, info = env.step(action)
    # print(obs)
    time.sleep(1/240)
    # husky_position, husky_orientation = p.getBasePositionAndOrientation(env.husky_id)
    # print(husky_position, husky_orientation)
    # Extract velocities
    joint_states = p.getJointStates(
        bodyUniqueId=env.panda_id,
        jointIndices=range(7)
    )

    velocities = [joint_state[1] for joint_state in joint_states]
    print(f"Joint Velocities: {velocities}")
    p.stepSimulation()

Joint Velocities: [-0.017292406757489567, 0.027324180516432484, 0.0013568897994054308, 0.0027773975901975584, 2.824905682280443e-05, 0.0, -0.0004823441815063889]
Joint Velocities: [0.004942469280101343, -0.007881588812975293, 0.0007960253160157363, -0.0016522061377139334, 7.465821012693644e-05, 5.149960319306146e-19, 0.0003980135419763268]
Joint Velocities: [0.02106027637673845, -0.021638159000684023, 0.004512179169981082, -0.0031406632244254033, 0.0001591738738144387, 8.131516293641283e-20, -7.853976419461476e-05]
Joint Velocities: [0.05052959044659096, -0.049032524857775185, 0.007649996608837307, -0.005626485770156075, 0.0002706079138438739, -1.3010426069826053e-18, -0.0008134992238501913]
Joint Velocities: [0.046408257842514025, -0.04176221521369204, 0.011313826588376203, -0.006069336367289627, 0.0003611095321336251, -4.87890977618477e-19, -0.0006444437004288409]
Joint Velocities: [0.042199908112412456, -0.038838499036292974, 0.009924953753373383, -0.00561842435460696, 0.00032325722